In [52]:
import numpy as np
from Session import Session
import pickle
import pandas as pd

In [53]:
def mrr( rank, label ):
    for i, item in enumerate(rank[:20]):
        if item == label:
            return 1.0/(i + 1)
    return 0

def recall( rank, label ):
    return int(label in rank[:20])
    
def pred2rank_result( pred ):
    top20_indices = np.argsort(pred)[-20:]
    top20_indices = np.flip(top20_indices, 0)
    rank_result = [ candidates[i] for i in top20_indices ]
    return rank_result


In [54]:
with open('../output/STAN_result.csv', 'r') as f:
    STAN_df = pd.read_csv(f)
with open('../output/SKNN_result.csv', 'r') as f:
    SKNN_df = pd.read_csv(f)
with open('../output/VSKNN_result.csv', 'r') as f:
    VSKNN_df = pd.read_csv(f)
with open( '../output/itemCF_result.csv', 'r') as f:
    itemCF_df = pd.read_csv(f)
with open('../output/NARM_100epochs.csv', 'r') as f:
    NARM_df = pd.read_csv(f)
    

In [55]:
len(STAN_df), len(SKNN_df), len(VSKNN_df), len(itemCF_df), len(NARM_df)

(2189125, 2154322, 2153915, 2196995, 277416)

In [56]:
STAN_df.session_id.nunique(), SKNN_df.session_id.nunique(), VSKNN_df.session_id.nunique(), itemCF_df.session_id.nunique(), NARM_df.session_id.nunique()

(55763, 55763, 55763, 55725, 55898)

In [77]:
session_set = set(STAN_df.session_id) & set(SKNN_df.session_id) & set(VSKNN_df.session_id) & set(itemCF_df.session_id) & set(NARM_df.session_id)
x_test = []
y_test = []
for i in session_set:
    x_test.append( session_ids[i] )
    y_test.append( labels )

In [79]:
STAN_df

,session_id,item_id,score
0,0,6409,146.212538
1,0,37169,30.326207
2,0,3644,25.904686
3,0,9878,18.231854
4,0,37146,16.778757
...,...,...,...
2189120,55897,37169,0.338093
2189121,55897,37204,0.338093
2189122,55897,117,0.228783
2189123,55897,37172,0.223772


In [85]:
STAN_df.columns

Index(['session_id', ' item_id', ' score'], dtype='object')

In [84]:
STAN_df.loc[STAN_df.session_id == 0, ['item_id']]

KeyError: "None of [Index(['item_id'], dtype='object')] are in the [columns]"

In [ ]:
for res_df in [STAN_df, SKNN_df, VSKNN_df, itemCF_df, NARM_df]:
    for x, y in zip(x_test, y_test):
        rank = res_df[res_df.session_id == x].item_id.values
        mrr = mrr( pred2rank_result(rank), y )